In [87]:
import gzip
import pandas as pd
import numpy as np
import pickle
import requests
import logging
import time
import random

f = open("token_TMDB.txt","r")
token = f.read()
headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}

In [88]:
movies = pickle.load(open('movies.pkl', 'rb'))

In [106]:
movies_mini = movies.head(10)

In [125]:
def fill_missing_data(df, headers):
    logger = logging.getLogger(__name__)
    updated_df = df.copy()  
    
    if 'keywords' not in updated_df.columns:
        updated_df['keywords'] = None
    if 'overview' not in updated_df.columns:
        updated_df['overview'] = None
    if 'production_countries' not in updated_df.columns:
        updated_df['production_countries'] = None
    if 'production_companies' not in updated_df.columns:
        updated_df['production_companies'] = None
    
    total_rows = len(updated_df)
    processed_rows = 0
    
    for index, row in updated_df.iterrows():
        imdb_id = row['tconst']
        url = f"https://api.themoviedb.org/3/find/{imdb_id}?external_source=imdb_id"
        
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            details = response.json()
            movie_results = details.get('movie_results', [])
            
            if not movie_results:
                logger.warning(f"No movie results found for IMDb ID: {imdb_id}. Skipping.")
                continue
            
            tmdb_id = movie_results[0]['id']
            api_url = f"https://api.themoviedb.org/3/movie/{tmdb_id}?append_to_response=keywords"
            
            try:
                response2 = requests.get(api_url, headers=headers)
                response2.raise_for_status()
                data = response2.json()
                
                if data.get('imdb_id') != imdb_id:
                    logger.warning(f"IMDb ID mismatch for movie with TMDb ID {tmdb_id}. Skipping.")
                    continue
                
                updated_df.at[index, 'keywords'] = data.get('keywords', [])
                updated_df.at[index, 'overview'] = data.get('overview', '')
                updated_df.at[index, 'production_countries'] = data.get('production_countries', [])
                updated_df.at[index, 'production_companies'] = data.get('production_companies', [])
                
            except requests.exceptions.RequestException as e:
                logger.error(f"Error processing URL {api_url}: {e}")
        
        except requests.exceptions.RequestException as e:
            logger.error(f"Error occurred while fetching data for IMDb ID {imdb_id}: {e}")
        
        processed_rows += 1
        progress = (processed_rows / total_rows) * 100
        
        print(f"Progress: {progress:.2f}% loading...", end="\r")
        
        request_interval = 1 / 50  # 50 requests per second is the TMDB limit

        if progress % 5 == 0:
            time.sleep(request_interval)
        else:
            sleep_duration = random.uniform(0.1 * request_interval, 0.5 * request_interval)
            time.sleep(sleep_duration)
 
    print("Progress: 100.00% loading... Completed.")
    
    return updated_df

logging.basicConfig(level=logging.INFO)


In [126]:
movies.columns

Index(['tconst', 'primaryTitle', 'originalTitle', 'startYear',
       'runtimeMinutes', 'genres'],
      dtype='object')

In [127]:
%%time
df = fill_missing_data(movies_mini, headers)

Progress: 100.00% loading... Completed.
CPU times: user 512 ms, sys: 22.9 ms, total: 535 ms
Wall time: 1.25 s


In [128]:
df.isna().sum()

tconst                  0
primaryTitle            0
originalTitle           0
startYear               0
runtimeMinutes          0
genres                  0
keywords                0
overview                0
production_countries    0
production_companies    0
dtype: int64

In [129]:
df1.to_pickle('movies2.pkl')

In [130]:
%%time
df1 = fill_missing_data(movies, headers)

Progress: 100.00% loading... Completed.
CPU times: user 59min 2s, sys: 3min 6s, total: 1h 2min 8s
Wall time: 3h 25min 42s


In [131]:
df1

tconst              primaryTitle                   originalTitle  \
28768     tt0029284          My Favorite Wife                My Favorite Wife   
30799     tt0031359                  Gaslight                        Gaslight   
31405     tt0031976       The Stars Look Down             The Stars Look Down   
31600     tt0032179          21 Days Together                         21 Days   
31602     tt0032181   Abe Lincoln in Illinois         Abe Lincoln in Illinois   
...             ...                       ...                             ...   
10698589  tt9907782                The Cursed                Eight for Silver   
10698877  tt9908390                   Le lion                         Le lion   
10700132  tt9911196       The Marriage Escape  De beentjes van Sint-Hildegard   
10702488  tt9916270  Il talento del calabrone        Il talento del calabrone   
10702533  tt9916362                     Coven                        Akelarre   

          startYear  runtimeMinutes                   genres  \
28768          1940              88           Comedy,Romance   
30799          1940              84         Mystery,Thriller   
31405          1940             110                    Drama   
31600          1940              72      Crime,Drama,Romance   
31602          1940             110  Biography,Drama,History   
...             ...             ...                      ...   
10698589       2021             111   Fantasy,Horror,Mystery   
10698877       2020              95                   Comedy   
10700132       2020             103             Comedy,Drama   
10702488       2020              84                 Thriller   
10702533       2020              92            Drama,History   

                                                   keywords  \
28768     {'keywords': [{'id': 931, 'name': 'jealousy'},...   
30799     {'keywords': [{'id': 6038, 'name': 'marriage'}...   
31405     {'keywords': [{'id': 6346, 'name': 'trade unio...   
31600     {'keywords': [{'id': 171989, 'name': 'wrongful...   
31602     {'keywords': [{'id': 5565, 'name': 'biography'...   
...                                                     ...   
10698589  {'keywords': [{'id': 394, 'name': 'gypsy'}, {'...   
10698877  {'keywords': [{'id': 5265, 'name': 'espionage'}]}   
10700132                                   {'keywords': []}   
10702488                                   {'keywords': []}   
10702533  {'keywords': [{'id': 344, 'name': 'inquisition...   

                                                   overview  \
28768     Years after she was presumed dead in a shipwre...   
30799     Twenty years removed from Alice Barlow's murde...   
31405     Davey Fenwick leaves his mining village on a u...   
31600     After Larry Darrent accidentally kills his lov...   
31602     Abe Lincoln in Illinois is a 1940 biographical...   
...                                                     ...   
10698589  In the late 19th century, a brutal land baron ...   
10698877  A psychiatric hospital patient pretends to be ...   
10700132  Jan has been married to Gedda for 35 years. Ge...   
10702488  Dj Steph is a young radio deejay on the rise, ...   
10702533  French Basque Country, year 1609. The men of a...   

                                       production_countries  \
28768     [{'iso_3166_1': 'US', 'name': 'United States o...   
30799      [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]   
31405      [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]   
31600      [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]   
31602     [{'iso_3166_1': 'US', 'name': 'United States o...   
...                                                     ...   
10698589  [{'iso_3166_1': 'US', 'name': 'United States o...   
10698877           [{'iso_3166_1': 'FR', 'name': 'France'}]   
10700132      [{'iso_3166_1': 'NL', 'name': 'Netherlands'}]   
10702488            [{'iso_3166_1': 'IT', 'name': 'Italy'}]   
10702533  [{'iso_3166_1': 'AR', 'name': 'Argentina'}, {'...   



In [132]:
df1.to_pickle('movies2.pkl')